In [14]:
import numpy as np

from bokeh.plotting import figure, show, output_file, output_notebook
from bokeh.palettes import viridis
from bokeh.layouts import gridplot
## show inline in the notebook
output_notebook()

TOOLS="hover,crosshair,pan,wheel_zoom,box_zoom,undo,redo,reset,tap,save"#,box_select,poly_select,lasso_select"
p = figure(tools=TOOLS,title="Energy Density", toolbar_location="above",
           plot_width=600, plot_height=600)

Loading BokehJS ...

In [48]:
# Compare to top part of case2phi.png
# I really tried to combine the top and bottom plots.
# Load in data:
# Data is stored as z,x,phi1,phi2,phi3,phi4,phi5,phi6. Want to print atan(phi3/phi1). atan2 is fully defined inverse polar coords.
data1 = np.loadtxt('./hw_2_data/fort.700')
data2 = np.loadtxt('./hw_2_data/fort.730')
data3 = np.loadtxt('./hw_2_data/fort.760')

n = 64
# Data1:
X1 = data1[:,0]
Y1 = data1[:,1]
U1 = data1[:,2]
V1 = data1[:,4]
scale1 = 10
length1 = np.sqrt(U1*U1 + V1*V1)*scale1+1
angle1 = np.arctan(V1/U1)
X11 = X1 + length1*np.cos(angle1)
Y11 = Y1 + length1*np.sin(angle1)

# Data2:
X2 = data2[:,0]
Y2 = data2[:,1]
U2 = data2[:,2]
V2 = data2[:,4]
scale2 = 10
length2 = np.sqrt(U2*U2 + V2*V2)*scale2+1
angle2 = np.arctan(V2/U2)
X12 = X2 + length2*np.cos(angle2)
Y12 = Y2 + length2*np.sin(angle2)

#Data3:
X3 = data3[:,0]
Y3 = data3[:,1]
U3 = data3[:,2]
V3 = data3[:,4]
scale3 = 10
length3 = np.sqrt(U3*U3 + V3*V3)*scale3+1
angle3 = np.arctan(V3/U3)
X13 = X3 + length3*np.cos(angle3)
Y13 = Y3 + length3*np.sin(angle3)


# Number of colours
n_col = 10
cm = viridis(n_col+1)#np.array(["#C7E9B4", "#7FCDBB", "#41B6C4", "#1D91C0", "#225EA8", "#0C2C84"])
ix = ((length-length.min())/(length.max()-length.min())*n_col).astype('int')
colors = [cm[i] for i in ix]

p1 = figure(x_range=(-n,n ), y_range=(-n, n),tools=TOOLS,title="Initial", toolbar_location="above",)
p1.segment(X1, Y1, X11, Y11, color=colors, line_width=2)
p2 = figure(x_range=(-n,n ), y_range=(-n, n), tools=TOOLS,title="Intermediate", toolbar_location="above",)
p2.segment(X2, Y2, X12, Y12, color=colors, line_width=2)
p3 = figure(x_range=(-n,n ), y_range=(-n, n), tools=TOOLS,title="Final", toolbar_location="above",)
p3.segment(X3, Y3, X13, Y13, color=colors, line_width=2)

output_file("phi_field.html", title="Replicating phi_field plot from paper")
show(gridplot([[p1,p2,p3]], plot_width=300, plot_height=300))

# Ignore below this

In [3]:
#######################################################################################
# This was my attempt to use my own data, but because contour plots don't work this wasn't amazing.
#######################################################################################
# Load in data:
# Column 0 = xdata
# Column 1 = ydata
# Column 2 = Magnetic Energy Density
# Column 3 = Potential Energy Density
# Column 4 = Scalar Energy Density
#data1 = np.loadtxt('./hw_2_data/case2mp.501')
#data2 = np.loadtxt('./hw_2_data/case2mp.512')
#data3 = np.loadtxt('./hw_2_data/case2mp.519')

#x = data1[:,0]
#y = data1[:,2]

#radii = 1.#data1[:,3]
#colors = viridis(len(data1[:,0])/20)
#colors = ["#%02x%02x%02x" % (int(r), int(g), 150) for r, g in zip(data1[:,2], data1[:,2])]

#p.scatter(x, y)#, color = colors)

#output_file("color_scatter.html", title="color_scatter.py example")

#show(p)  # open a browser

In [38]:
#######################################################################################
# This was my attempt to use my own data, but because contour plots don't work this wasn't amazing.
#######################################################################################
import numpy as np

from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_file

def streamlines(x, y, u, v, density=1):
    ''' Return streamlines of a vector flow.

    * x and y are 1d arrays defining an *evenly spaced* grid.
    * u and v are 2d arrays (shape [y,x]) giving velocities.
    * density controls the closeness of the streamlines. For different
      densities in each direction, use a tuple or list [densityx, densityy].

    '''

    ## Set up some constants - size of the grid used.
    NGX = len(x)
    NGY = len(y)

    ## Constants used to convert between grid index coords and user coords.
    DX = x[1]-x[0]
    DY = y[1]-y[0]
    XOFF = x[0]
    YOFF = y[0]

    ## Now rescale velocity onto axes-coordinates
    u = u / (x[-1]-x[0])
    v = v / (y[-1]-y[0])
    speed = np.sqrt(u*u+v*v)
    ## s (path length) will now be in axes-coordinates, but we must
    ## rescale u for integrations.
    u *= NGX
    v *= NGY
    ## Now u and v in grid-coordinates.

    NBX = int(30*density)
    NBY = int(30*density)
    blank = np.zeros((NBY,NBX))

    bx_spacing = NGX/float(NBX-1)
    by_spacing = NGY/float(NBY-1)

    def blank_pos(xi, yi):
        return int((xi / bx_spacing) + 0.5), \
               int((yi / by_spacing) + 0.5)

    def value_at(a, xi, yi):
        if type(xi) == np.ndarray:
            x = xi.astype(np.int)
            y = yi.astype(np.int)
        else:
            x = np.int(xi)
            y = np.int(yi)
        a00 = a[y,x]
        a01 = a[y,x+1]
        a10 = a[y+1,x]
        a11 = a[y+1,x+1]
        xt = xi - x
        yt = yi - y
        a0 = a00*(1-xt) + a01*xt
        a1 = a10*(1-xt) + a11*xt
        return a0*(1-yt) + a1*yt

    def rk4_integrate(x0, y0):
        ## This function does RK4 forward and back trajectories from
        ## the initial conditions, with the odd 'blank array'
        ## termination conditions. TODO tidy the integration loops.

        def f(xi, yi):
            dt_ds = 1./value_at(speed, xi, yi)
            ui = value_at(u, xi, yi)
            vi = value_at(v, xi, yi)
            return ui*dt_ds, vi*dt_ds

        def g(xi, yi):
            dt_ds = 1./value_at(speed, xi, yi)
            ui = value_at(u, xi, yi)
            vi = value_at(v, xi, yi)
            return -ui*dt_ds, -vi*dt_ds

        check = lambda xi, yi: xi>=0 and xi<NGX-1 and yi>=0 and yi<NGY-1

        bx_changes = []
        by_changes = []

        ## Integrator function
        def rk4(x0, y0, f):
            ds = 0.01 #min(1./NGX, 1./NGY, 0.01)
            stotal = 0
            xi = x0
            yi = y0
            xb, yb = blank_pos(xi, yi)
            xf_traj = []
            yf_traj = []
            while check(xi, yi):
                # Time step. First save the point.
                xf_traj.append(xi)
                yf_traj.append(yi)
                # Next, advance one using RK4
                try:
                    k1x, k1y = f(xi, yi)
                    k2x, k2y = f(xi + .5*ds*k1x, yi + .5*ds*k1y)
                    k3x, k3y = f(xi + .5*ds*k2x, yi + .5*ds*k2y)
                    k4x, k4y = f(xi + ds*k3x, yi + ds*k3y)
                except IndexError:
                    # Out of the domain on one of the intermediate steps
                    break
                xi += ds*(k1x+2*k2x+2*k3x+k4x) / 6.
                yi += ds*(k1y+2*k2y+2*k3y+k4y) / 6.
                # Final position might be out of the domain
                if not check(xi, yi): break
                stotal += ds
                # Next, if s gets to thres, check blank.
                new_xb, new_yb = blank_pos(xi, yi)
                if new_xb != xb or new_yb != yb:
                    # New square, so check and colour. Quit if required.
                    if blank[new_yb,new_xb] == 0:
                        blank[new_yb,new_xb] = 1
                        bx_changes.append(new_xb)
                        by_changes.append(new_yb)
                        xb = new_xb
                        yb = new_yb
                    else:
                        break
                if stotal > 2:
                    break
            return stotal, xf_traj, yf_traj

        integrator = rk4

        sf, xf_traj, yf_traj = integrator(x0, y0, f)
        sb, xb_traj, yb_traj = integrator(x0, y0, g)
        stotal = sf + sb
        x_traj = xb_traj[::-1] + xf_traj[1:]
        y_traj = yb_traj[::-1] + yf_traj[1:]

        ## Tests to check length of traj. Remember, s in units of axes.
        if len(x_traj) < 1: return None
        if stotal > .2:
            initxb, inityb = blank_pos(x0, y0)
            blank[inityb, initxb] = 1
            return x_traj, y_traj
        else:
            for xb, yb in zip(bx_changes, by_changes):
                blank[yb, xb] = 0
            return None

    ## A quick function for integrating trajectories if blank==0.
    trajectories = []
    def traj(xb, yb):
        if xb < 0 or xb >= NBX or yb < 0 or yb >= NBY:
            return
        if blank[yb, xb] == 0:
            t = rk4_integrate(xb*bx_spacing, yb*by_spacing)
            if t is not None:
                trajectories.append(t)

    ## Now we build up the trajectory set. I've found it best to look
    ## for blank==0 along the edges first, and work inwards.
    for indent in range((max(NBX,NBY))//2):
        for xi in range(max(NBX,NBY)-2*indent):
            traj(xi+indent, indent)
            traj(xi+indent, NBY-1-indent)
            traj(indent, xi+indent)
            traj(NBX-1-indent, xi+indent)

    xs = [np.array(t[0])*DX+XOFF for t in trajectories]
    ys = [np.array(t[1])*DY+YOFF for t in trajectories]

    return xs, ys

xx = np.linspace(-3, 3, 100)
yy = np.linspace(-3, 3, 100)
# Load in data:
# Data is stored as z,x,phi1,phi2,phi3,phi4,phi5,phi6. Want to print atan(phi3/phi1). atan2 is fully defined inverse polar coords.
data1 = np.loadtxt('./hw_2_data/fort.701')
#data2 = np.loadtxt('./hw_2_data/fort.712')
#data3 = np.loadtxt('./hw_2_data/fort.719')

#xx = data1[:,0]
n = 64
X = data1[:,0]
Y = data1[:,1]
#Y, X = np.meshgrid(xx, yy)
U = data1[:,2]
V = data1[:,4]
#speed = np.sqrt(U*U + V*V)
length = np.sqrt(U*U + V*V)
#theta = np.arctan(V/U)
angle = np.arctan(V/U)

#x0 = X[::2, ::2].flatten()
#y0 = Y[::2, ::2].flatten()
#length = speed[::2, ::2].flatten()/40
#angle = theta[::2, ::2].flatten()
x1 = x0 + length * np.cos(angle)
y1 = y0 + length * np.sin(angle)

#xs, ys = streamlines(xx, yy, U.T, V.T, density=2)

# Number of colours
n_col = 10
cm = viridis()#np.array(["#C7E9B4", "#7FCDBB", "#41B6C4", "#1D91C0", "#225EA8", "#0C2C84"])
ix = ((length-length.min())/(length.max()-length.min())*n_col).astype('int')
colors = cm[ix]

p1 = figure(x_range=(-64,64 ), y_range=(-64, 64))
p1.segment(x0, y0, x1, y1, color=colors, line_width=2)

#p2 = figure(x_range=(-64,64), y_range=(-64,64))
#p2.multi_line(xs, ys, color="#ee6666", line_width=2, line_alpha=0.8)

output_file("vector.html", title="vector.py example")

show(p1)


IndexError: too many indices for array

In [ ]:
import numpy as np

from bokeh.models import ColumnDataSource, DataRange1d, Plot, LinearAxis, Grid
from bokeh.models.glyphs import Segment
from bokeh.io import curdoc, show

N = 9
x = np.linspace(-2, 2, N)
y = x**2

source = ColumnDataSource(dict(
        x=x,
        y=y,
        xm01=x-x**3/10 + 0.3,
        ym01=y-x**2/10 + 0.5,
    )
)

xdr = DataRange1d()
ydr = DataRange1d()

plot = Plot(
    title=None, x_range=xdr, y_range=ydr, plot_width=300, plot_height=300,
    h_symmetry=False, v_symmetry=False, min_border=0, toolbar_location=None)

glyph = Segment(x0="x", y0="y", x1="xm01", y1="ym01", line_color="#f4a582", line_width=3)
plot.add_glyph(source, glyph)

xaxis = LinearAxis()
plot.add_layout(xaxis, 'below')

yaxis = LinearAxis()
plot.add_layout(yaxis, 'left')

plot.add_layout(Grid(dimension=0, ticker=xaxis.ticker))
plot.add_layout(Grid(dimension=1, ticker=yaxis.ticker))

curdoc().add_root(plot)

show(plot)